# 銀河のぼんやり白いもの（Colab版 / v2）

Colabでそのまま動くようにした版です。  
**回帰 → カウント（ポアソン） → 不確実性 → 外れ値 →（おまけで）PyTorch**まで、ひと続きで試せます。

- `u`：銀河帯の位置（中心に近いほど“厚い”想定）
- `overlap`：レンズの重なり（鮮明さ）
- `star_count`：見える星の数（カウント）
- `haze_obs`：ぼんやり白さ（連続値）
- `memo`：本文由来の観測メモ（雰囲気担当）

> 合言葉：深呼吸してから検証する（まず観測条件を疑う）


In [ ]:
# Colab向け：必要なものをインストール
# （Colabには大体入っていますが、バージョン差が出ないように明示します）
!pip -q install -U scikit-learn matplotlib plotly

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, PoissonRegressor, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_poisson_deviance

import plotly.express as px
import plotly.graph_objects as go


## 1) データの用意

次のどれかでOKです：

- **A. リポジトリを Colab で開いてる場合**：`data/ginga_galaxy_haze_v2.csv` があれば自動で読みます  
- **B. 手元のCSVをアップロード**：次のセルでアップロードして読みます  
- **C. GitHubのrawから取る**：URLを書き換えて `pd.read_csv(url)` で読みます（サンプル用）


In [ ]:
import os
from google.colab import files

# A) repo内にある想定パス（必要に応じて変えてOK）
CANDIDATES = [
    "data/ginga_galaxy_haze_v2.csv",
    "ginga_galaxy_haze_v2.csv",
    "ginga_galaxy_haze_v2_sample.csv",
    "data/sample/ginga_galaxy_haze_v2_sample.csv",
]

csv_path = None
for p in CANDIDATES:
    if os.path.exists(p):
        csv_path = p
        break

if csv_path is None:
    print("CSVが見つかりません。アップロードします（A/B/CのB）。")
    up = files.upload()  # ここでCSVを選ぶ
    csv_path = list(up.keys())[0]

print("Using:", csv_path)
df = pd.read_csv(csv_path)
df.head(5)


## 2) まず眺める（窓景色の“統計”）

- 分布（ヒストグラム）
- `u` と `overlap` と目的変数の関係（散布図）


In [ ]:
# ヒストグラム（matplotlib）
fig, ax = plt.subplots()
ax.hist(df["star_count"], bins=40)
ax.set_title("star_count (見える星の数)")
plt.show()

fig, ax = plt.subplots()
ax.hist(df["haze_obs"], bins=40)
ax.set_title("haze_obs (ぼんやり白さ)")
plt.show()

fig, ax = plt.subplots()
ax.hist(df["overlap"], bins=30)
ax.set_title("overlap (レンズの重なり)")
plt.show()


In [ ]:
# インタラクティブ散布図（plotly）
# u vs haze_obs（色＝overlap）
fig = px.scatter(df, x="u", y="haze_obs", color="overlap",
                 hover_data=["star_count","memo"],
                 title="u vs haze_obs (color=overlap)")
fig.show()

# u vs star_count（色＝overlap）
fig = px.scatter(df, x="u", y="star_count", color="overlap",
                 hover_data=["haze_obs","memo"],
                 title="u vs star_count (color=overlap)")
fig.show()


## 3) 特徴量づくり：中心ほど厚い → `u^2` を入れる

In [ ]:
df2 = df.copy()
df2["u2"] = df2["u"]**2

base_cols = ["u","u2","overlap","depth_blue","f_white","f_blue","f_light","f_star","f_len"]
X_base = df2[base_cols]


## 4) ぼんやり白さ（連続）を回帰で説明する

まずはシンプルに線形回帰。  
係数を見て「何が白さに効いてそう？」を物語っぽく言葉にしてみます。


In [ ]:
y = df2["haze_obs"]
Xtr, Xte, ytr, yte = train_test_split(X_base, y, test_size=0.25, random_state=0)

lin = LinearRegression().fit(Xtr, ytr)
pred = lin.predict(Xte)
print("LinearRegression MSE (haze):", mean_squared_error(yte, pred))

coef = pd.Series(lin.coef_, index=X_base.columns).sort_values(ascending=False)
coef


## 5) 交互作用：青×白（depth_blueの活用）＋多重共線性の気配

「青さが深いと、白いぼんやりが打ち消される/変形されるのでは？」という仮説で、交互作用を入れてみます。

- `depth_blue * f_white`
- `depth_blue * overlap`
- `overlap * f_white`

増やすと楽しいけど、解釈が難しくなったり推定が不安定になったりするのも論点。


In [ ]:
df2["blue_x_white"] = df2["depth_blue"] * df2["f_white"]
df2["blue_x_overlap"] = df2["depth_blue"] * df2["overlap"]
df2["overlap_x_white"] = df2["overlap"] * df2["f_white"]

cols = base_cols + ["blue_x_white","blue_x_overlap","overlap_x_white"]
X = df2[cols]
y = df2["haze_obs"]

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=0)

lin2 = LinearRegression().fit(Xtr, ytr)
pred2 = lin2.predict(Xte)
print("LinearRegression MSE (haze, +interaction):", mean_squared_error(yte, pred2))

pd.Series(lin2.coef_, index=X.columns).sort_values(ascending=False)


In [ ]:
# 相関（ざっくり）と条件数（大きいほど不安定になりがち）
corr = pd.DataFrame(X).corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(7,6))
im = ax.imshow(corr.to_numpy(), vmin=-1, vmax=1)
ax.set_xticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=90, fontsize=8)
ax.set_yticks(range(len(corr.columns)))
ax.set_yticklabels(corr.columns, fontsize=8)
ax.set_title("Correlation (features)")
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.show()

Xs = StandardScaler().fit_transform(X)
cond = np.linalg.cond(Xs)
print("condition number:", cond)


In [ ]:
# Lasso（選ぶ）と Ridge（安定化）で比較
lasso = make_pipeline(StandardScaler(), LassoCV(cv=5, random_state=0, max_iter=20000))
lasso.fit(Xtr, ytr)
pred_l = lasso.predict(Xte)
print("Lasso MSE:", mean_squared_error(yte, pred_l))

ridge = make_pipeline(StandardScaler(), RidgeCV(alphas=np.logspace(-3,3,30), cv=5))
ridge.fit(Xtr, ytr)
pred_r = ridge.predict(Xte)
print("Ridge MSE:", mean_squared_error(yte, pred_r))

coef_l = pd.Series(lasso.named_steps["lassocv"].coef_, index=X.columns).sort_values(ascending=False)
coef_l


## 6) 「ぼんやり」＝観測ノイズ：overlapで分散が変わるか？

overlapが小さい（レンズが重ならない）ほど、滲みが増えて観測が不安定になるはず。


In [ ]:
tmp = df2.copy()
tmp["over_bin"] = pd.qcut(tmp["overlap"], q=6, duplicates="drop")
g = tmp.groupby("over_bin")["haze_obs"].agg(["mean","var","count"]).reset_index()
g


## 7) 星の数（カウント）：線形回帰 vs ポアソン回帰（GLM）

カウントは「非負の整数」。  
ポアソン回帰は **平均が上がると揺らぎも増える**世界に自然にフィットします。

※ PoissonRegressor が重い場合があるので、学習はサブサンプルします。


In [ ]:
Xc = df2[cols]  # 交互作用も含めてみる
y_cnt = df2["star_count"]

Xtr, Xte, ytr, yte = train_test_split(Xc, y_cnt, test_size=0.25, random_state=0)

lin_cnt = LinearRegression().fit(Xtr, ytr)
pred_lin = lin_cnt.predict(Xte)
print("Linear MSE (count):", mean_squared_error(yte, pred_lin))
print("neg prediction rate:", float((pred_lin < 0).mean()))

sub = 900
if len(Xtr) > sub:
    Xtr_p = Xtr.sample(sub, random_state=0)
    ytr_p = ytr.loc[Xtr_p.index]
else:
    Xtr_p, ytr_p = Xtr, ytr

pois = PoissonRegressor(alpha=1e-4, max_iter=1000).fit(Xtr_p, ytr_p)
pred_p = pois.predict(Xte)
print("Poisson deviance:", mean_poisson_deviance(yte, pred_p))

pd.Series(pois.coef_, index=Xc.columns).sort_values(ascending=False)


In [ ]:
# mean-variance の手触り（uでビン分け）
tmp = df2.copy()
tmp["u_bin"] = pd.qcut(tmp["u"], q=10, duplicates="drop")
mv = tmp.groupby("u_bin")["star_count"].agg(["mean","var","count"]).reset_index()
mv


## 8) 外れ値（Outlier）を見つけて“物語として読む”

v2データには、意図的に「強い外れ値」を混ぜてある場合があります（公開用サンプルとしての仕掛け）。  
公開用なら event 列は基本隠したまま残差から探すのが楽しいです。

ここではポアソン回帰の期待値 λ を使って、標準化残差を計算します。


In [ ]:
# 全データに対して残差を計算（poissonモデルでの期待値）
mu = pois.predict(Xc)  # 期待値（λ）
res = (df2["star_count"].to_numpy() - mu) / np.sqrt(np.clip(mu, 1e-6, None))

# 外れ値トップ10
idx = np.argsort(np.abs(res))[::-1][:10]
out = df2.loc[idx, ["obs_id","star_count","u","overlap","haze_obs","memo"]].copy()
out["std_residual"] = res[idx]
out


In [ ]:
# 外れ値を散布図でハイライト（plotly）
df_plot = df2.copy()
df_plot["std_residual"] = res
df_plot["is_outlier"] = (np.abs(res) > np.quantile(np.abs(res), 0.995)).astype(int)

fig = px.scatter(df_plot, x="u", y="star_count", color="is_outlier",
                 hover_data=["overlap","haze_obs","std_residual","memo"],
                 title="Outliers by standardized residual (Poisson)")
fig.show()


## 9) 不確実性つき予測（GP）：白さに“幅”をつける

GPは「どれくらい分からないか（標準偏差）」も返してくれる回帰です。  
ここでは軽くするためにサブサンプルします。

さらに、overlap が小さいときにノイズが増えるので、  
overlapビンごとの分散をざっくり見積もって `alpha` に入れます（雑だけど教育的）。


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel

# サブサンプル（GPはO(n^3)）
n_gp = 450
sub = df2.sample(min(n_gp, len(df2)), random_state=0).copy()

Xgp = sub[["u","overlap"]].to_numpy()
ygp = sub["haze_obs"].to_numpy()

sub["over_bin"] = pd.qcut(sub["overlap"], q=6, duplicates="drop")
var_by_bin = sub.groupby("over_bin")["haze_obs"].var().to_dict()
alpha = sub["over_bin"].map(var_by_bin).fillna(sub["haze_obs"].var()).to_numpy()
alpha = np.clip(alpha, 1e-6, None)

kernel = ConstantKernel(1.0) * RBF(length_scale=0.6) + WhiteKernel(noise_level=1e-2)
gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, normalize_y=True, random_state=0)
gp.fit(Xgp, ygp)
print("learned kernel:", gp.kernel_)

u_grid = np.linspace(df2["u"].min(), df2["u"].max(), 250)

def plot_slice(overlap_value):
    Xq = np.column_stack([u_grid, np.full_like(u_grid, overlap_value)])
    mu, std = gp.predict(Xq, return_std=True)

    fig, ax = plt.subplots()
    ax.plot(u_grid, mu, label="mean")
    ax.fill_between(u_grid, mu - 2*std, mu + 2*std, alpha=0.2, label="±2σ")
    ax.set_title(f"GP: haze_obs (overlap={overlap_value:.2f})")
    ax.set_xlabel("u")
    ax.set_ylabel("haze_obs")
    ax.legend()
    plt.show()

plot_slice(0.85)
plot_slice(0.15)


In [ ]:
# GPの“面”を軽く可視化（plotly）
# 低解像度でOK
u_lin = np.linspace(df2["u"].min(), df2["u"].max(), 60)
o_lin = np.linspace(0.0, 1.0, 45)
U, O = np.meshgrid(u_lin, o_lin)
Xq = np.column_stack([U.ravel(), O.ravel()])
mu, std = gp.predict(Xq, return_std=True)
MU = mu.reshape(O.shape)
SD = std.reshape(O.shape)

fig = go.Figure(data=[go.Surface(x=U, y=O, z=MU)])
fig.update_layout(title="GP mean surface: haze_obs(u, overlap)",
                  scene=dict(xaxis_title="u", yaxis_title="overlap", zaxis_title="mean haze_obs"),
                  height=600)
fig.show()

fig = go.Figure(data=[go.Surface(x=U, y=O, z=SD)])
fig.update_layout(title="GP std surface: uncertainty(u, overlap)",
                  scene=dict(xaxis_title="u", yaxis_title="overlap", zaxis_title="std"),
                  height=600)
fig.show()


## 10) おまけ：PyTorchでポアソン（tabular）

「次はD2Lへ」という橋渡し用。  
Colabなら torch はだいたい入っています。


In [ ]:
try:
    import torch
    import torch.nn as nn
    from torch.utils.data import TensorDataset, DataLoader

    d = df2.sample(min(1200, len(df2)), random_state=0).copy()
    feats = ["u","u2","overlap","depth_blue","f_white","f_blue","f_light","f_star","f_len"]
    X_t = torch.tensor(d[feats].to_numpy(), dtype=torch.float32)
    y_t = torch.tensor(d["star_count"].to_numpy(), dtype=torch.float32).view(-1,1)

    ds = TensorDataset(X_t, y_t)
    dl = DataLoader(ds, batch_size=64, shuffle=True)

    model = nn.Linear(X_t.shape[1], 1)
    loss_fn = nn.PoissonNLLLoss(log_input=False, full=True)
    opt = torch.optim.Adam(model.parameters(), lr=1e-2)

    for epoch in range(8):
        total = 0.0
        for xb, yb in dl:
            lam = torch.exp(model(xb)) + 1e-6
            loss = loss_fn(lam, yb)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total += loss.item() * len(xb)
        print("epoch", epoch, "loss", total/len(ds))

    w = model.weight.detach().cpu().numpy().ravel()
    b = float(model.bias.detach().cpu().numpy().ravel()[0])
    print("bias", b)
    print(dict(zip(feats, w)))

except Exception as e:
    print("torch not available or failed:", e)


## メモ（ゆるめ）

- 線形回帰とポアソン回帰、どっちが「銀河っぽい」説明になる？
- overlap が小さいとき、GPの“幅”が増える？（増えてたらOK）
- 交互作用を入れると、何が分かりやすくなって、何が難しくなる？
- 外れ値は消す？読む？（ここが楽しい）
- もし星図が画像なら：CNN＋どんな損失にする？

以上！